<a href="https://colab.research.google.com/github/beyzaturku/2209/blob/main/medium_alt_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Klasör yolları
image_val_dir = "/content/drive/MyDrive/medium_alt_object_detection/images/val"
label_val_dir = "/content/drive/MyDrive/medium_alt_object_detection/labels/val"

# Train klasöründeki tüm görüntülerin isimlerini al (sadece isim kısmını al)
image_files = {os.path.splitext(f)[0] for f in os.listdir(image_val_dir)}
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_val_dir)}

# Ortak olan dosyalar
common_files = image_files.intersection(label_files)
missing_labels = image_files - label_files
missing_images = label_files - image_files

# Sonuçları yazdır
print(f"📂 Toplam {len(image_files)} görüntü dosyası, {len(label_files)} etiket dosyası var.")
print(f"✅ Eşleşen dosya sayısı: {len(common_files)}")

if missing_labels:
    print(f"⚠️ {len(missing_labels)} görüntü dosyasının etiketi eksik!")
    print("\nEksik etiketler:")
    for file in list(missing_labels)[:10]:  # İlk 10 eksik dosyayı göster
        print(file + ".txt")

if missing_images:
    print(f"⚠️ {len(missing_images)} etiket dosyasının görüntüsü eksik!")
    print("\nEksik görüntüler:")
    for file in list(missing_images)[:10]:  # İlk 10 eksik dosyayı göster
        print(file + ".jpg")

📂 Toplam 3227 görüntü dosyası, 3227 etiket dosyası var.
✅ Eşleşen dosya sayısı: 3227


### YOLO'ya uygun olması için data.yaml dosyası oluşturulur

In [10]:
yaml_content = """train: /content/drive/MyDrive/medium_alt_object_detection/images/train
val: /content/drive/MyDrive/medium_alt_object_detection/images/val

nc: 4  # Added space after 'nc'
names: ["car", "vehicle", "truck", "bus"]
"""

yaml_path = "/content/drive/MyDrive/medium_alt_object_detection/data.yaml"

with open(yaml_path, "w") as file:
    file.write(yaml_content)

print(f"✅ data.yaml başarıyla kaydedildi: {yaml_path}")

✅ data.yaml başarıyla kaydedildi: /content/drive/MyDrive/medium_alt_object_detection/data.yaml


In [6]:
!cat /content/drive/MyDrive/medium_alt_object_detection/data.yaml

train: /content/drive/MyDrive/medium_alt_object_detection/images/train
val:/content/drive/MyDrive/medium_alt_object_detection/images/val

nc: 4
names: ["car", "vehicle", "truck", "bus"]


### Model Eğitimi

In [7]:
!pip install ultralytics
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [11]:
from ultralytics import YOLO

# Modeli oluştur
model = YOLO("yolov8n.pt")

# Modeli eğit
results = model.train(
    data="/content/drive/MyDrive/medium_alt_object_detection/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    project="/content/drive/MyDrive/medium_alt_object_detection",
    name="yolov8_medium_alt",
    save=True,
    save_period=5,
    val=True
)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/medium_alt_object_detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/drive/MyDrive/medium_alt_object_detection, name=yolov8_medium_alt, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

100%|██████████| 755k/755k [00:00<00:00, 22.2MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 84.3MB/s]


AMP: checks passed ✅


...

      50/50      2.71G     0.5973     0.3067     0.7907        183        640: 100%|██████████| 807/807 [01:25<00:00,  9.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:14<00:00,  6.95it/s]

                   all       3227      55268      0.988      0.985      0.994      0.873



50 epochs completed in 1.467 hours.
Optimizer stripped from /content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:22<00:00,  4.50it/s]


                   all       3227      55268      0.988      0.985      0.994      0.873
                   car       3110      51384       0.99      0.971      0.994      0.845
                 truck       1175       2372      0.984      0.984      0.994      0.862
                   bus        656       1512      0.989      0.999      0.995      0.911
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt


### Doğrulama seti ile test etme

In [12]:
# Eğitilmiş modeli yükle
best_model = YOLO("/content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt/weights/best.pt")

# Modelin performansını doğrulama setinde test et
metrics = best_model.val()
print(metrics)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/medium_alt_object_detection/labels/val.cache... 3227 images, 67 backgrounds, 0 corrupt: 100%|██████████| 3227/3227 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 202/202 [00:51<00:00,  3.89it/s]


                   all       3227      55268      0.988      0.985      0.994      0.877
                   car       3110      51384      0.991      0.971      0.994      0.852
                 truck       1175       2372      0.984      0.984      0.994      0.866
                   bus        656       1512      0.989      0.999      0.995      0.913
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e39547fad50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.

In [13]:
import pandas as pd
import matplotlib.pyplot as plt

# YOLOv8 sonuçlarını içeren CSV dosyasının yolu
results_path = '/content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt/results.csv'

# CSV dosyasını oku
results_df = pd.read_csv(results_path)

print(results_df)

    epoch      time  train/box_loss  train/cls_loss  train/dfl_loss  \
0       1   118.028         1.52696         1.44491         0.94405   
1       2   228.469         1.37235         0.85741         0.89978   
2       3   337.655         1.33581         0.76303         0.89340   
3       4   444.923         1.26308         0.68761         0.88126   
4       5   551.839         1.17704         0.61637         0.86720   
5       6   658.591         1.12086         0.57480         0.85727   
6       7   766.040         1.07811         0.54712         0.85063   
7       8   872.316         1.04445         0.52545         0.84463   
8       9   978.681         1.02080         0.51140         0.84142   
9      10  1085.140         0.99450         0.49750         0.83739   
10     11  1191.040         0.96849         0.48411         0.83430   
11     12  1297.230         0.95209         0.47443         0.83184   
12     13  1403.490         0.93363         0.46430         0.82967   
13    

In [27]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Modeli yükle
model = YOLO("/content/drive/MyDrive/medium_alt_object_detection/yolov8_medium_alt/weights/best.pt")

# Tahmin yap
results = model("/content/drive/MyDrive/medium_alt_object_detection/images/val/M0603_img000094.jpg", save=True, conf=0.01, verbose=True)

# Çıktı görüntüsünün yolu (varsayılan olarak "runs/detect/predict/" içine kaydedilir)
output_image_path = "/content/runs/detect/predict"

img = cv2.imread(output_image_path)
# Dosyayı okuyup okuyamadığınızı kontrol edin
if img is None:
    print(f"Error: Could not read image from {output_image_path}. Please check the path and file existence.")
else:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV BGR formatında okuduğu için RGB'ye çevir

    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.show()


image 1/1 /content/drive/MyDrive/medium_alt_object_detection/images/val/M0603_img000094.jpg: 352x640 13 cars, 1 truck, 2 buss, 8.4ms
Speed: 2.1ms preprocess, 8.4ms inference, 1.6ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs/detect/predict8
Error: Could not read image from /content/runs/detect/predict. Please check the path and file existence.


In [23]:
from PIL import Image

try:
    img = Image.open("/content/drive/MyDrive/medium_alt_object_detection/images/val/M0401_img000092.jpg")
    img.show()
    print("Görüntü başarıyla yüklendi.")
except Exception as e:
    print(f"Görüntü yükleme hatası: {e}")


Görüntü başarıyla yüklendi.
